## 以程式方式使用 LLM

在 Generative AI 興起的這個年代，許多人肯定與 ChatGPT 這種大型語言模型(LLM, Large Language Mode)互動過，且一般是透過 UI 或應用程式來完成。

這個範例中，將使用 Python 透過其 API 直接連接與查詢已部署的 LLM，而本次 Lab 採用 **Mistral-7B Instruct v2**。(https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) 模型，這是一個開源模型(Apache 2.0 license)，雖然比其他商業或開源模型輕量，但卻有很好的功能。

P.S. 此模型預設已經部署到 Lab 叢集上，雖然是輕量的模型，但依然需要具備 24GB RAM 的 GPU 才能執行。

### Requirements and Imports

如果有依照前面流程啟用 Workbench Image 的話，預設都會包含所需 Libraries，因此可以直接 Import 使用。但如果未使用正確 Image 的話，則需要執行第一行 `pip install ...` 來安裝相依套件。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain

LangChain 是開放原始碼的架構，可根據大型語言模型 (LLM) 打造應用程式。LLM 是針對大量資料預先訓練的大型深度學習模型，可以對使用者查詢生成回應，例如回答問題或從文本提示中建立影像。LangChain 提供工具和抽象化，藉以改善模型產生之資訊的客製化程度、準確度和關聯性。例如，開發人員可使用 LangChain 元件建立新提示鏈，或客製化現有的範本。LangChain 還包括其他元件，讓 LLM 無須重新訓練即可取得新資料。

下面將建立一個 **llm** 實例，該實例由可以查詢 LLM API 的位置以及將應用於模型的一些參數來定義。e.g. `max_new_tokens` 定義模型最多使用 512 tokens(單字或單字的一部分)進行回答; 而 `temperature` 則表示模型回答接近事實(數值越低越接近)。

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

接著還需要建立一個 **template** 來應用於將發送到模型的每個請求(即 "Prompt")。

基於此，在查詢模型時，可以透過以下只是來提供模型給出正確的回答，例如以下情境:

* 你(模型)是一位樂於助人、受人尊敬且誠實的助手。 在確保安全的同時，始終盡可能提供協助
* 你(模型)的答案不應包含任何有害、不道德、種族主義、性別歧視、有毒、危險或非法內容。

In [ ]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchain 現在允許將這些元素「縫合」在一起，並建立一個用來查詢模型的 **conversation** 物件。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

現在可以帶入查詢資訊來查看結果。

In [ ]:
query = "什麼是 AI?"

conversation.predict(input=query); # ";" at the end of the line hides final output (repetion of the streamed answer)

後續將在第 3.5 節回到此 Notebook 進行一些練習，因此可以暫時將其開著。